## 📈 Predicting Premier League Final Positions Using Betting Odds, Probabilistic Modelling & Simulation

**Competition:** English Rugby Premiership 2025/26  
**Purpose:** Estimate probabilities of final league positions using betting market information and simulation  
**Methods:** Odds-implied probabilities, Monte Carlo simulation, scenario analysis  
**Author:** [Victoria Friss de Kereki](https://www.linkedin.com/in/victoria-friss-de-kereki/) 
**Medium Article:** [Predicting Premier League Final Positions Using Betting Odds, Probabilistic Modelling & Simulation](https://medium.com/p/2720ec335c3c)

---

**Notebook first written:** `17/01/2026`  
**Last updated:** `26/01/2026`  

> This notebook develops a probabilistic framework to predict final English Rugby Premiership final positions using betting odds as market-based expectations.
>
> Betting odds are transformed into implied probabilities and adjusted for bookmaker margin. These probabilities are then used to simulate the remainder of the season via Monte Carlo methods, generating distributions over final points totals and league positions.
>
> The analysis focuses on estimating the likelihood of key outcomes such as title wins, top-four finishes, relegation, and mid-table placements. Results are presented at team level with uncertainty intervals, and the framework can be extended to incorporate form, fixture difficulty, or alternative predictive inputs beyond betting markets.


<div style="text-align: left;">
    <img src="Images and others for Medium/Predicting PREMIERSHIP final positions.png"  
         alt="Predicting PREMIERSHIP final positions"  
         width="600">
</div>

In [1]:
# import soccerdata as sd

# Core
from datetime import datetime, timedelta
import os

# Data manipulation
import numpy as np
import pandas as pd

# APIs & environment
import requests
from dotenv import load_dotenv

# Statistics
from scipy.stats import poisson

# Visualisation
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

## 1. Premiership Final Standings (ESPN Scraping)
##### Adapting the ESPN scraper I built in my previous project to be used for Rugby Premiership instead of Football Premier League.

In [2]:
url = "https://www.espn.com/rugby/standings/_/league/267979"
tables = pd.read_html(url)

# Fix ESPN header issue
teams_raw = tables[0].copy()

# Move column names into first row
teams_raw.loc[-1] = teams_raw.columns
teams_raw.index = teams_raw.index + 1
teams_raw = teams_raw.sort_index().reset_index(drop=True)

# Stats table is fine as-is
stats = tables[1]

# Parse team table
teams = pd.DataFrame()

teams["position"] = (
    teams_raw.iloc[:, 0]
    .astype(str)
    .str.extract(r"^(\d+)")
    .astype(int)
)

teams["team"] = (
    teams_raw.iloc[:, 0]
    .astype(str)
    .str.replace(r"^\d+", "", regex=True)        # remove position
    .str.replace(r"^[A-Z]{2,3}", "", regex=True) # remove abbreviation
    .str.strip()
)

# Parse stats table
stats.columns = [
    "gp", "w", "d", "l", "bye",
    "pf", "pa", "tf", "ta",
    "tbp", "lbp", "bp",
    "pd", "pts"
]

stats = stats.apply(
    lambda c: (
        c.astype(str)
         .str.replace("+", "", regex=False)
         .astype(int)
    )
)

# Combine
premiership = pd.concat([teams, stats], axis=1)

premiership

,position,team,gp,w,d,l,bye,pf,pa,tf,ta,tbp,lbp,bp,pd,pts
0,1,Northampton Saints,10,8,1,1,0,390,282,58,39,9,0,9,108,43
1,2,Bath Rugby,10,8,0,2,0,343,237,50,34,8,1,9,106,41
2,3,Bristol Rugby,10,8,0,2,0,290,235,41,34,5,0,5,55,37
3,4,Leicester Tigers,10,7,0,3,0,301,238,41,31,7,1,8,63,36
4,5,Exeter Chiefs,10,6,1,3,0,272,179,37,24,6,3,9,93,35
5,6,Saracens,10,5,0,5,0,383,248,56,35,9,3,12,135,32
6,7,Sale Sharks,10,3,0,7,0,285,297,39,43,5,3,8,-12,20
7,8,Gloucester Rugby,10,1,0,9,0,214,335,32,45,4,3,7,-121,11
8,9,Harlequins,10,2,0,8,0,196,351,26,52,2,0,2,-155,10
9,10,Newcastle Falcons,10,1,0,9,0,167,439,23,66,1,0,1,-272,5


## 2. Get betting odds using API

In [3]:
# Load variables from API_KEY.env
load_dotenv("API_KEY.env")

API_KEY = os.getenv("ODDS_DATA_API_KEY")

if API_KEY is None:
    raise ValueError("API_KEY not found. Check API_KEY.env")

print("API key loaded successfully")

API key loaded successfully


In [4]:
url = "https://api.the-odds-api.com/v4/sports"
params = {"apiKey": API_KEY}

sports = requests.get(url, params=params).json()

[r for r in sports if "rugby" in r["key"].lower()]

[{'key': 'rugbyleague_nrl',
  'group': 'Rugby League',
  'title': 'NRL',
  'description': 'Aussie Rugby League',
  'active': True,
  'has_outrights': False},
 {'key': 'rugbyleague_nrl_state_of_origin',
  'group': 'Rugby League',
  'title': 'State of Origin',
  'description': 'State of Origin series',
  'active': True,
  'has_outrights': False},
 {'key': 'rugbyunion_six_nations',
  'group': 'Rugby Union',
  'title': 'Six Nations',
  'description': 'Six Nations Championship',
  'active': True,
  'has_outrights': False}]

In [4]:
url = "https://api.the-odds-api.com/v4/sports/soccer_epl/odds"

params = {
    "apiKey": API_KEY,
    "regions": "uk",
    "markets": "h2h",
    "oddsFormat": "decimal",
    "dateFormat": "iso",
    "days": 365  # get all upcoming matches for the next year
}

response = requests.get(url, params=params)
response.raise_for_status()

odds_data = response.json()
print("Total upcoming matches:", len(odds_data))

Total upcoming matches: 21


In [5]:
def flatten_odds(data):
    rows = []

    for match in data:
        match_id = match["id"]
        home = match["home_team"]
        away = match["away_team"]
        time = match["commence_time"]

        for book in match["bookmakers"]:
            bookmaker = book["title"]

            # Find head-to-head (h2h) market. Find the market where key == 'h2h' (win/draw/win odds). If not found, skip this bookmaker.
            h2h = next((m for m in book["markets"] if m["key"] == "h2h"), None)
            if not h2h:
                continue

            outcomes = {o["name"]: o["price"] for o in h2h["outcomes"]}

            rows.append({
                "match_id": match_id,
                "commence_time": time,
                "home_team": home,
                "away_team": away,
                "bookmaker": bookmaker,
                "home_odds": outcomes.get(home),
                "draw_odds": outcomes.get("Draw"),
                "away_odds": outcomes.get(away),
            })

    return pd.DataFrame(rows)

df = flatten_odds(odds_data)
df.head()

,match_id,commence_time,home_team,away_team,bookmaker,home_odds,draw_odds,away_odds
0,e90e22b348a38af568f940b4b5d92ca6,2026-01-26T20:00:00Z,Everton,Leeds United,Unibet (UK),2.55,3.2,2.90
1,e90e22b348a38af568f940b4b5d92ca6,2026-01-26T20:00:00Z,Everton,Leeds United,Paddy Power,2.50,3.1,2.88
2,e90e22b348a38af568f940b4b5d92ca6,2026-01-26T20:00:00Z,Everton,Leeds United,Betway,2.50,3.2,2.88
3,e90e22b348a38af568f940b4b5d92ca6,2026-01-26T20:00:00Z,Everton,Leeds United,Smarkets,2.66,3.3,3.05
4,e90e22b348a38af568f940b4b5d92ca6,2026-01-26T20:00:00Z,Everton,Leeds United,Sky Bet,2.50,3.2,2.88


In [76]:
betting_odds_avg = (
    df.groupby(["match_id", "home_team", "away_team"])
      .agg({
          "home_odds": "mean",
          "draw_odds": "mean",
          "away_odds": "mean"
      })
      .reset_index()
)

betting_odds_avg.head()

,match_id,home_team,away_team,home_odds,draw_odds,away_odds
0,0232baa88f2ea7539654e9f8ebd53272,Arsenal,Sunderland,1.235000,5.635714,12.750000
1,36820753efb36739a83c6e5e440827b2,Brighton and Hove Albion,Everton,1.812778,3.636111,4.182222
2,38a3cb5e295f55e274d589fc646cf2dd,Tottenham Hotspur,Manchester City,4.643750,4.168750,1.646250
3,4ae3a1134fb1d10167c21bf10dff498f,Leeds United,Nottingham Forest,2.268571,3.253571,3.110714
4,5064501681485a3db4abd297a0e775f4,Fulham,Everton,1.962143,3.292857,3.910714


In [77]:
# Convert odds to implied probabilities
prob_cols = {
    "p_home_raw": 1 / betting_odds_avg["home_odds"],
    "p_draw_raw": 1 / betting_odds_avg["draw_odds"],
    "p_away_raw": 1 / betting_odds_avg["away_odds"],
}

betting_odds_avg = betting_odds_avg.assign(**prob_cols)

# Remove bookmaker margin (normalise)
total = (
    betting_odds_avg["p_home_raw"] +
    betting_odds_avg["p_draw_raw"] +
    betting_odds_avg["p_away_raw"]
)

betting_odds_avg = betting_odds_avg.assign(
    p_home_book=betting_odds_avg["p_home_raw"] / total,
    p_draw_book=betting_odds_avg["p_draw_raw"] / total,
    p_away_book=betting_odds_avg["p_away_raw"] / total,
)

# Keep only required fields
betting_odds_avg = betting_odds_avg[
    [
        "home_team",
        "away_team",
        "p_home_book",
        "p_draw_book",
        "p_away_book",
    ]
]

betting_odds_avg.head()


,home_team,away_team,p_home_book,p_draw_book,p_away_book
0,Arsenal,Sunderland,0.759878,0.166518,0.073604
1,Brighton and Hove Albion,Everton,0.517599,0.258048,0.224353
2,Tottenham Hotspur,Manchester City,0.202645,0.225735,0.571621
3,Leeds United,Nottingham Forest,0.412111,0.287347,0.300543
4,Fulham,Everton,0.476732,0.284074,0.239193


## 3. Get fixtures for upcoming EPL games
#### No API available for Premiership Rugby, so performing web scraping from https://www.skysports.com/rugby-union/competitions/gallagher-prem/fixtures instead.

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.skysports.com/rugby-union/competitions/gallagher-prem/fixtures"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

fixtures_container = soup.find("div", class_="fixres__body")

month = None
date = None

dates = []
months = []
home_teams = []
away_teams = []
times = []

for elem in fixtures_container.children:
    # Skip empty elements or strings
    if not hasattr(elem, "name"):
        continue

    # Month header
    if elem.name == "h3" and "fixres__header1" in elem.get("class", []):
        month = elem.get_text(strip=True)

    # Date header
    elif elem.name == "h4" and "fixres__header2" in elem.get("class", []):
        date = elem.get_text(strip=True)

    # Match item
    elif elem.name == "div" and "fixres__item" in elem.get("class", []):
        home = elem.select_one("span.matches__participant--side1 span.swap-text__target").get_text(strip=True)
        away = elem.select_one("span.matches__participant--side2 span.swap-text__target").get_text(strip=True)
        time = elem.select_one("span.matches__date").get_text(strip=True)

        months.append(month)
        dates.append(date)
        home_teams.append(home)
        away_teams.append(away)
        times.append(time)

# Build DataFrame
df = pd.DataFrame({
    "month": months,
    "date": dates,
    "time": times,
    "home_team": home_teams,
    "away_team": away_teams
})

print(df.head())
print("Total fixtures scraped:", len(df))

        month                 date   time         home_team  \
0  March 2026    Friday 20th March  19:45              Bath   
1  March 2026  Saturday 21st March  15:00        Harlequins   
2  March 2026  Saturday 21st March  15:00       Northampton   
3  March 2026  Saturday 21st March  15:05            Exeter   
4  March 2026    Sunday 22nd March  15:00  Leicester Tigers   

             away_team  
0             Saracens  
1           Gloucester  
2  Newcastle Red Bulls  
3          Sale Sharks  
4        Bristol Bears  
Total fixtures scraped: 43


In [7]:
# Remove fixtures with TBC in either team
df_clean = df[~df["home_team"].str.contains("TBC") & ~df["away_team"].str.contains("TBC")].reset_index(drop=True)

print(df_clean.head())
print("Total fixtures after removing TBC matches:", len(df_clean))

        month                 date   time         home_team  \
0  March 2026    Friday 20th March  19:45              Bath   
1  March 2026  Saturday 21st March  15:00        Harlequins   
2  March 2026  Saturday 21st March  15:00       Northampton   
3  March 2026  Saturday 21st March  15:05            Exeter   
4  March 2026    Sunday 22nd March  15:00  Leicester Tigers   

             away_team  
0             Saracens  
1           Gloucester  
2  Newcastle Red Bulls  
3          Sale Sharks  
4        Bristol Bears  
Total fixtures after removing TBC matches: 40


In [8]:
df_clean

,month,date,time,home_team,away_team
0,March 2026,Friday 20th March,19:45,Bath,Saracens
1,March 2026,Saturday 21st March,15:00,Harlequins,Gloucester
2,March 2026,Saturday 21st March,15:00,Northampton,Newcastle Red Bulls
3,March 2026,Saturday 21st March,15:05,Exeter,Sale Sharks
4,March 2026,Sunday 22nd March,15:00,Leicester Tigers,Bristol Bears
5,March 2026,Friday 27th March,19:45,Newcastle Red Bulls,Exeter
6,March 2026,Saturday 28th March,13:00,Gloucester,Leicester Tigers
7,March 2026,Saturday 28th March,15:30,Bristol Bears,Harlequins
8,March 2026,Saturday 28th March,18:00,Saracens,Northampton
9,March 2026,Sunday 29th March,15:00,Sale Sharks,Bath


## 4. Get this season (2025/26) and last season (2024/25) results
#### Again, no API available here. Will do web scraping from https://www.skysports.com/rugby-union/competitions/gallagher-prem/results/2025-26 and previous seasons.

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_sky_sports_results(url, season_label):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    
    container = soup.find("div", class_="fixres__body")
    if not container:
        print(f"No results container found at {url}")
        return pd.DataFrame()
    
    month = None
    date_text = None
    
    rows = []
    
    for elem in container.children:
        if not hasattr(elem, "name"):
            continue
        
        # Month header
        if elem.name == "h3" and "fixres__header1" in elem.get("class", []):
            month = elem.get_text(strip=True)
        
        # Date header
        elif elem.name == "h4" and "fixres__header2" in elem.get("class", []):
            date_text = elem.get_text(strip=True)
        
        # Match item
        elif elem.name == "div" and "fixres__item" in elem.get("class", []):
            home = elem.select_one("span.matches__participant--side1 span.swap-text__target")
            away = elem.select_one("span.matches__participant--side2 span.swap-text__target")
            
            # Scores appear in spans with class matches__teamscores-side
            scores = elem.select("span.matches__teamscores-side")
            if len(scores) >= 2:
                home_score = scores[0].get_text(strip=True)
                away_score = scores[1].get_text(strip=True)
            else:
                # If no score found, treat as n/a
                home_score = None
                away_score = None
            
            # Clean text
            home_team = home.get_text(strip=True) if home else None
            away_team = away.get_text(strip=True) if away else None
            
            # Combine date and month into full date string
            full_date_str = f"{date_text} {month}"
            # Attempt to parse
            try:
                match_date = datetime.strptime(full_date_str, "%A %dth %B %Y")
            except Exception:
                try:
                    match_date = datetime.strptime(full_date_str, "%A %dnd %B %Y")
                except Exception:
                    match_date = full_date_str
            
            rows.append({
                "season": season_label,
                "date": match_date,
                "home_team": home_team,
                "home_score": home_score,
                "away_team": away_team,
                "away_score": away_score
            })
    
    return pd.DataFrame(rows)


# URLs and season labels
urls = [
    ("https://www.skysports.com/rugby-union/competitions/gallagher-prem/results/2025-26", "2025-26"),
    ("https://www.skysports.com/rugby-union/competitions/gallagher-prem/results/2024-25", "2024-25"),
    ("https://www.skysports.com/rugby-union/competitions/gallagher-prem/results/2023-24", "2023-24"),
]

# Scrape all
all_results = pd.concat(
    scrape_sky_sports_results(url, season) for url, season in urls
).reset_index(drop=True)

print(all_results.head())
print("Total past results:", len(all_results))


    season                                date    home_team home_score  \
0  2025-26  Saturday 24th January January 2026       Exeter          3   
1  2025-26  Saturday 24th January January 2026     Saracens         73   
2  2025-26  Saturday 24th January January 2026   Harlequins          7   
3  2025-26  Saturday 24th January January 2026  Sale Sharks         29   
4  2025-26    Friday 23rd January January 2026   Gloucester         26   

             away_team away_score  
0        Bristol Bears          8  
1  Newcastle Red Bulls         14  
2     Leicester Tigers         34  
3          Northampton         43  
4                 Bath         30  
Total past results: 236


In [13]:
import pandas as pd
from datetime import datetime

def clean_match_dates(df):
    # Remove duplicate month if it exists in the day string
    df['date_clean'] = df['date'].str.replace(r'(\b\w+)\s(\1\s\d{4})', r'\2', regex=True)
    
    # Try parsing to datetime
    def parse_date(d):
        try:
            return datetime.strptime(d, "%A %dth %B %Y")
        except:
            try:
                return datetime.strptime(d, "%A %dnd %B %Y")
            except:
                try:
                    return datetime.strptime(d, "%A %dst %B %Y")
                except:
                    try:
                        return datetime.strptime(d, "%A %drd %B %Y")
                    except:
                        return d  # fallback: leave as string
    
    df['date'] = df['date_clean'].apply(parse_date)
    df.drop(columns=['date_clean'], inplace=True)
    return df

# Apply to your past results DataFrame
all_results_fixed = clean_match_dates(all_results)
all_results_fixed.head()

,season,date,home_team,home_score,away_team,away_score
0,2025-26,2026-01-24,Exeter,3,Bristol Bears,8
1,2025-26,2026-01-24,Saracens,73,Newcastle Red Bulls,14
2,2025-26,2026-01-24,Harlequins,7,Leicester Tigers,34
3,2025-26,2026-01-24,Sale Sharks,29,Northampton,43
4,2025-26,2026-01-23,Gloucester,26,Bath,30


In [23]:
# Sort by season and date just in case
all_results_fixed = all_results_fixed.sort_values(["season", "date"]).reset_index(drop=True)

# Seasons to remove playoffs from (previous 2 seasons)
previous_seasons = ['2024-25', '2023-24']

# Function to remove last N matches per season for specific seasons
def remove_playoffs_specific(df, seasons, playoff_count=3):
    result = []
    for season, group in df.groupby("season"):
        if season in seasons:
            result.append(group.iloc[:-playoff_count])
        else:
            result.append(group)
    return pd.concat(result).reset_index(drop=True)

# Apply
all_results_no_playoffs = remove_playoffs_specific(all_results_fixed, previous_seasons, playoff_count=3)

print("Total matches after removing playoffs from previous seasons:", len(all_results_no_playoffs))

Total matches after removing playoffs from previous seasons: 230


In [24]:
all_results_no_playoffs

,season,date,home_team,home_score,away_team,away_score
0,2023-24,2023-10-13,Bristol Bears,25,Leicester Tigers,14
1,2023-24,2023-10-14,Exeter,65,Saracens,10
2,2023-24,2023-10-14,Bath,34,Newcastle Red Bulls,26
3,2023-24,2023-10-14,Gloucester,29,Harlequins,28
4,2023-24,2023-10-15,Sale Sharks,20,Northampton,15
...,...,...,...,...,...,...
225,2025-26,2026-01-23,Gloucester,26,Bath,30
226,2025-26,2026-01-24,Exeter,3,Bristol Bears,8
227,2025-26,2026-01-24,Saracens,73,Newcastle Red Bulls,14
228,2025-26,2026-01-24,Harlequins,7,Leicester Tigers,34


In [26]:
# Sort by date just to be safe
all_results_no_playoffs = all_results_no_playoffs.sort_values(["season", "date"]).reset_index(drop=True)

# Identify last 5 matches of the current season
current_season = '2025-26'
mask = all_results_no_playoffs['season'] == current_season
current_season_df = all_results_no_playoffs[mask]

# Drop last 5 matches of this season
trimmed_current_season = current_season_df.iloc[:-5]

# Keep all other seasons unchanged
all_other_seasons = all_results_no_playoffs[all_results_no_playoffs['season'] != current_season]

# Combine back
all_results_balanced = pd.concat([all_other_seasons, trimmed_current_season]).sort_values(["season", "date"]).reset_index(drop=True)

# Quick check
print("Total matches after trimming last 5 of current season:", len(all_results_balanced))

Total matches after trimming last 5 of current season: 225


In [28]:
all_results_balanced.tail()

,season,date,home_team,home_score,away_team,away_score
220,2025-26,2026-01-02,Bristol Bears,19,Sale Sharks,17
221,2025-26,2026-01-02,Newcastle Red Bulls,25,Gloucester,19
222,2025-26,2026-01-03,Bath,33,Exeter,26
223,2025-26,2026-01-03,Northampton,66,Harlequins,21
224,2025-26,2026-01-04,Leicester Tigers,36,Saracens,28


## 5. Combine and calculate probabilities of W/D/L for each match

In [17]:
# Load Dataframes
df_current = past_matches_25_clean
df_prev = past_matches_24_clean
df_future = df_fixtures_clean

# Combine all past fixtures together
df_all = pd.concat([df_prev, df_current], ignore_index=True)

In [18]:
# Add weights: more recent games = more weight
df_all["date"] = pd.to_datetime(df_all["utcDate"])
df_all["weight"] = np.linspace(1, 2, len(df_all))  # simple linear weighting

In [19]:
df_all.tail()

,utcDate,matchday,status,homeTeam,awayTeam,homeGoals,awayGoals,winner,date,weight
604,2026-01-24T17:30:00Z,23,FINISHED,AFC Bournemouth,Liverpool FC,3,2,HOME_TEAM,2026-01-24 17:30:00+00:00,1.993421
605,2026-01-25T14:00:00Z,23,FINISHED,Crystal Palace FC,Chelsea FC,1,3,AWAY_TEAM,2026-01-25 14:00:00+00:00,1.995066
606,2026-01-25T14:00:00Z,23,FINISHED,Brentford FC,Nottingham Forest FC,0,2,AWAY_TEAM,2026-01-25 14:00:00+00:00,1.996711
607,2026-01-25T14:00:00Z,23,FINISHED,Newcastle United FC,Aston Villa FC,0,2,AWAY_TEAM,2026-01-25 14:00:00+00:00,1.998355
608,2026-01-25T16:30:00Z,23,FINISHED,Arsenal FC,Manchester United FC,2,3,AWAY_TEAM,2026-01-25 16:30:00+00:00,2.000000


In [20]:
# Compute home advantage
# Home advantage = average home goals - average away goals
home_advantage = (
    df_all["homeGoals"].mean() - df_all["awayGoals"].mean()
)
home_advantage

0.1888341543513956

In [21]:
#  Calculate attack & defense strengths
teams = pd.unique(df_all[["homeTeam", "awayTeam"]].values.ravel("K"))

attack = pd.Series(1.0, index=teams)
defense = pd.Series(1.0, index=teams)

# Initialize with goals per match
team_stats = {}

for team in teams:
    home_games = df_all[df_all["homeTeam"] == team]
    away_games = df_all[df_all["awayTeam"] == team]

    goals_scored = (home_games["homeGoals"] * home_games["weight"]).sum() + \
                   (away_games["awayGoals"] * away_games["weight"]).sum()

    goals_against = (home_games["awayGoals"] * home_games["weight"]).sum() + \
                    (away_games["homeGoals"] * away_games["weight"]).sum()

    matches = home_games["weight"].sum() + away_games["weight"].sum()

    team_stats[team] = {
        "scored": goals_scored / matches,
        "against": goals_against / matches
    }

# Strengths = relative to league average
league_avg_scored = df_all["homeGoals"].mean() + df_all["awayGoals"].mean()
league_avg_scored /= 2

for team in teams:
    attack[team] = team_stats[team]["scored"] / league_avg_scored
    defense[team] = team_stats[team]["against"] / league_avg_scored

🔥 Summary

This function:
+ Calculates expected goals for each team
+ Uses Poisson distribution to compute goal probabilities
+ Converts score probabilities into match outcome probabilities
+ Returns probabilities for:
++ home win
++ draw
++ away win

The Poisson distribution models the number of goals a team scores in a match based on an expected goal rate (λ). Using the formula \(P(X=k)=e^{-\lambda}\lambda^k/k!\), it calculates the probability of scoring 0, 1, 2, … goals, where λ is estimated from team attack/defense strengths and league averages. In the model, I compute separate Poisson probabilities for home and away goals, then combine them to get the probabilities of each possible scoreline and therefore the probabilities of a home win, draw, or away win.


In [22]:
# Calculate probabilities for each future match

def match_probabilities(home, away):
    # expected goals
    exp_home = np.exp(np.log(league_avg_scored) + np.log(attack[home]) + np.log(defense[away]) + home_advantage)
    exp_away = np.exp(np.log(league_avg_scored) + np.log(attack[away]) + np.log(defense[home]))

    # compute probabilities up to 6 goals (I tested this number, and 6 produces the smallest rmse when compare to the bookmaker odds)
    max_goals = 6
    p_home = poisson.pmf(range(max_goals + 1), exp_home)
    p_away = poisson.pmf(range(max_goals + 1), exp_away)

    # result probabilities
    p_win = 0
    p_draw = 0
    p_loss = 0

    for i in range(max_goals + 1):
        for j in range(max_goals + 1):
            prob = p_home[i] * p_away[j]
            if i > j:
                p_win += prob
            elif i == j:
                p_draw += prob
            else:
                p_loss += prob

    return p_win, p_draw, p_loss

In [23]:
# Apply to all fixtures

results = []

for _, row in df_future.iterrows():
    home = row["homeTeam"]
    away = row["awayTeam"]

    p_win, p_draw, p_loss = match_probabilities(home, away)

    results.append({
        "utcDate": row["utcDate"],
        "homeTeam": home,
        "awayTeam": away,
        "p_home_win": p_win,
        "p_draw": p_draw,
        "p_away_win": p_loss,
    })

df_odds = pd.DataFrame(results)
df_odds.head()


,utcDate,homeTeam,awayTeam,p_home_win,p_draw,p_away_win
0,2026-01-26T20:00:00Z,Everton FC,Leeds United FC,0.483798,0.252150,0.262731
1,2026-01-31T15:00:00Z,Brighton & Hove Albion FC,Everton FC,0.460659,0.254820,0.283332
2,2026-01-31T15:00:00Z,Leeds United FC,Arsenal FC,0.164150,0.200730,0.629051
3,2026-01-31T15:00:00Z,Wolverhampton Wanderers FC,AFC Bournemouth,0.264887,0.218474,0.511896
4,2026-01-31T17:30:00Z,Chelsea FC,West Ham United FC,0.696273,0.168438,0.122443


## 6. Compare calculated probabilities to bookmaker ones

In [24]:
unique_bet_home = betting_odds_avg["home_team"].unique()
unique_model_home = df_odds["homeTeam"].unique()

In [25]:
print(unique_bet_home)
print(unique_model_home)

['Arsenal' 'Brighton and Hove Albion' 'Tottenham Hotspur' 'Leeds United'
 'Fulham' 'Newcastle United' 'Sunderland' 'Manchester United' 'Liverpool'
 'Burnley' 'Bournemouth' 'Aston Villa' 'Chelsea' 'Wolverhampton Wanderers'
 'Nottingham Forest' 'Everton']
['Everton FC' 'Brighton & Hove Albion FC' 'Leeds United FC'
 'Wolverhampton Wanderers FC' 'Chelsea FC' 'Liverpool FC' 'Aston Villa FC'
 'Manchester United FC' 'Nottingham Forest FC' 'Tottenham Hotspur FC'
 'Sunderland AFC' 'AFC Bournemouth' 'Arsenal FC' 'Burnley FC' 'Fulham FC'
 'Newcastle United FC' 'West Ham United FC' 'Crystal Palace FC'
 'Manchester City FC' 'Brentford FC']


In [26]:
def normalize_team(name):
    name = name.lower()
    name = name.replace(" fc", "")
    name = name.replace(" afc", "")
    name = name.replace("&", "and")
    name = name.replace("afc ", "")   # <--- this removes AFC from start
    name = name.strip()
    return name


In [27]:
df_odds["home_norm"] = df_odds["homeTeam"].apply(normalize_team)
df_odds["away_norm"] = df_odds["awayTeam"].apply(normalize_team)

betting_odds_avg["home_norm"] = betting_odds_avg["home_team"].apply(normalize_team)
betting_odds_avg["away_norm"] = betting_odds_avg["away_team"].apply(normalize_team)


In [28]:
unique_model_norm = df_odds["home_norm"].unique()
unique_bet_norm = betting_odds_avg["home_norm"].unique()

set(unique_model_norm) == set(unique_bet_norm)

False

In [29]:
df_compare = df_odds.merge(
    betting_odds_avg,
    left_on=["home_norm", "away_norm"],
    right_on=["home_norm", "away_norm"],
    how="inner"
)

print("Matched rows:", len(df_compare))
df_compare.head()

Matched rows: 21


,utcDate,homeTeam,awayTeam,p_home_win,p_draw,p_away_win,home_norm,away_norm,home_team,away_team,p_home_book,p_draw_book,p_away_book
0,2026-01-26T20:00:00Z,Everton FC,Leeds United FC,0.483798,0.252150,0.262731,everton,leeds united,Everton,Leeds United,0.371258,0.301594,0.327148
1,2026-01-31T15:00:00Z,Brighton & Hove Albion FC,Everton FC,0.460659,0.254820,0.283332,brighton and hove albion,everton,Brighton and Hove Albion,Everton,0.517599,0.258048,0.224353
2,2026-01-31T15:00:00Z,Leeds United FC,Arsenal FC,0.164150,0.200730,0.629051,leeds united,arsenal,Leeds United,Arsenal,0.153816,0.231198,0.614986
3,2026-01-31T15:00:00Z,Wolverhampton Wanderers FC,AFC Bournemouth,0.264887,0.218474,0.511896,wolverhampton wanderers,bournemouth,Wolverhampton Wanderers,Bournemouth,0.301451,0.264680,0.433869
4,2026-01-31T17:30:00Z,Chelsea FC,West Ham United FC,0.696273,0.168438,0.122443,chelsea,west ham united,Chelsea,West Ham United,0.630500,0.210900,0.158600


In [30]:
df_compare["diff_home"] = df_compare["p_home_win"] - df_compare["p_home_book"]
df_compare["diff_draw"] = df_compare["p_draw"] - df_compare["p_draw_book"]
df_compare["diff_away"] = df_compare["p_away_win"] - df_compare["p_away_book"]

df_compare[["homeTeam", "awayTeam", "diff_home", "diff_draw", "diff_away"]].head()

,homeTeam,awayTeam,diff_home,diff_draw,diff_away
0,Everton FC,Leeds United FC,0.112540,-0.049444,-0.064416
1,Brighton & Hove Albion FC,Everton FC,-0.056940,-0.003229,0.058979
2,Leeds United FC,Arsenal FC,0.010334,-0.030468,0.014065
3,Wolverhampton Wanderers FC,AFC Bournemouth,-0.036564,-0.046206,0.078027
4,Chelsea FC,West Ham United FC,0.065773,-0.042462,-0.036157


In [31]:
rmse_home = np.sqrt(np.mean((df_compare["p_home_win"] - df_compare["p_home_book"])**2))
rmse_draw = np.sqrt(np.mean((df_compare["p_draw"] - df_compare["p_draw_book"])**2))
rmse_away = np.sqrt(np.mean((df_compare["p_away_win"] - df_compare["p_away_book"])**2))

rmse_home, rmse_draw, rmse_away

(0.06130100360451834, 0.03238908913099239, 0.05849730826503058)

In [32]:
rmse_total = np.sqrt(np.mean(
    (df_compare["p_home_win"] - df_compare["p_home_book"])**2 +
    (df_compare["p_draw"] - df_compare["p_draw_book"])**2 +
    (df_compare["p_away_win"] - df_compare["p_away_book"])**2
))

rmse_total

0.09071274007497819

> Note: RMSE_TOTAL varies often, as bookmaker odds vary. However, it's stayed between 0.075 and 0.090.

In [33]:
np.mean([rmse_home, rmse_draw, rmse_away])

0.05072913366684711

In [34]:
df_compare["abs_diff"] = (
    abs(df_compare["diff_home"]) +
    abs(df_compare["diff_draw"]) +
    abs(df_compare["diff_away"])
)

df_compare.sort_values("abs_diff", ascending=False).head(10)[
    ["homeTeam", "awayTeam", "diff_home", "diff_draw", "diff_away"]
]


,homeTeam,awayTeam,diff_home,diff_draw,diff_away
7,Manchester United FC,Fulham FC,-0.157733,0.015169,0.139856
0,Everton FC,Leeds United FC,0.112540,-0.049444,-0.064416
14,Arsenal FC,Sunderland AFC,-0.110203,0.059196,0.048838
12,Manchester United FC,Tottenham Hotspur FC,-0.079357,-0.021639,0.096106
3,Wolverhampton Wanderers FC,AFC Bournemouth,-0.036564,-0.046206,0.078027
4,Chelsea FC,West Ham United FC,0.065773,-0.042462,-0.036157
11,Leeds United FC,Nottingham Forest FC,-0.022336,-0.047343,0.067583
16,Fulham FC,Everton FC,-0.056924,-0.011574,0.067890
17,Wolverhampton Wanderers FC,Chelsea FC,-0.034818,-0.031813,0.061179
8,Nottingham Forest FC,Crystal Palace FC,-0.052890,-0.010203,0.062252


## 7. Replace my estimates probabilities with the ones I have from odds, creating my final match probabilities

In [35]:
df_odds.head(2)

,utcDate,homeTeam,awayTeam,p_home_win,p_draw,p_away_win,home_norm,away_norm
0,2026-01-26T20:00:00Z,Everton FC,Leeds United FC,0.483798,0.25215,0.262731,everton,leeds united
1,2026-01-31T15:00:00Z,Brighton & Hove Albion FC,Everton FC,0.460659,0.25482,0.283332,brighton and hove albion,everton


In [36]:
betting_odds_avg.head(2)

,home_team,away_team,p_home_book,p_draw_book,p_away_book,home_norm,away_norm
0,Arsenal,Sunderland,0.759878,0.166518,0.073604,arsenal,sunderland
1,Brighton and Hove Albion,Everton,0.517599,0.258048,0.224353,brighton and hove albion,everton


In [37]:
df_final_probabilities = df_odds.merge(
    betting_odds_avg,
    left_on=["home_norm", "away_norm"],
    right_on=["home_norm", "away_norm"],
    how="left"
)

In [38]:
df_final_probabilities = df_final_probabilities[[
    "utcDate",
    "homeTeam",
    "awayTeam",
    "p_home_win",
    "p_draw",
    "p_away_win",
    "p_home_book",
    "p_draw_book",
    "p_away_book",
]]

df_final_probabilities

,utcDate,homeTeam,awayTeam,p_home_win,p_draw,p_away_win,p_home_book,p_draw_book,p_away_book
0,2026-01-26T20:00:00Z,Everton FC,Leeds United FC,0.483798,0.252150,0.262731,0.371258,0.301594,0.327148
1,2026-01-31T15:00:00Z,Brighton & Hove Albion FC,Everton FC,0.460659,0.254820,0.283332,0.517599,0.258048,0.224353
2,2026-01-31T15:00:00Z,Leeds United FC,Arsenal FC,0.164150,0.200730,0.629051,0.153816,0.231198,0.614986
3,2026-01-31T15:00:00Z,Wolverhampton Wanderers FC,AFC Bournemouth,0.264887,0.218474,0.511896,0.301451,0.264680,0.433869
4,2026-01-31T17:30:00Z,Chelsea FC,West Ham United FC,0.696273,0.168438,0.122443,0.630500,0.210900,0.158600
...,...,...,...,...,...,...,...,...,...
146,2026-05-24T15:00:00Z,Liverpool FC,Brentford FC,0.564042,0.197011,0.228989,NaN,NaN,NaN
147,2026-05-24T15:00:00Z,Manchester City FC,Aston Villa FC,0.575068,0.215298,0.205142,NaN,NaN,NaN
148,2026-05-24T15:00:00Z,Nottingham Forest FC,AFC Bournemouth,0.417804,0.236658,0.343129,NaN,NaN,NaN
149,2026-05-24T15:00:00Z,Tottenham Hotspur FC,Everton FC,0.419584,0.257934,0.321449,NaN,NaN,NaN


In [39]:
df_final_probabilities["p_home_final"] = np.where(
    df_final_probabilities["p_home_book"].notna(),
    df_final_probabilities["p_home_book"],
    df_final_probabilities["p_home_win"]
)

df_final_probabilities["p_draw_final"] = np.where(
    df_final_probabilities["p_draw_book"].notna(),
    df_final_probabilities["p_draw_book"],
    df_final_probabilities["p_draw"]
)

df_final_probabilities["p_away_final"] = np.where(
    df_final_probabilities["p_away_book"].notna(),
    df_final_probabilities["p_away_book"],
    df_final_probabilities["p_away_win"]
)

In [40]:
print("Used betting odds:", df_final_probabilities["p_home_book"].notna().sum())
print("Used model:", df_final_probabilities["p_home_book"].isna().sum())


Used betting odds: 21
Used model: 130


In [41]:
df_final_probabilities = df_final_probabilities[[
    "utcDate",
    "homeTeam",
    "awayTeam",
    "p_home_final",
    "p_draw_final",
    "p_away_final"
]]

In [42]:
df_final_probabilities

,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-01-26T20:00:00Z,Everton FC,Leeds United FC,0.371258,0.301594,0.327148
1,2026-01-31T15:00:00Z,Brighton & Hove Albion FC,Everton FC,0.517599,0.258048,0.224353
2,2026-01-31T15:00:00Z,Leeds United FC,Arsenal FC,0.153816,0.231198,0.614986
3,2026-01-31T15:00:00Z,Wolverhampton Wanderers FC,AFC Bournemouth,0.301451,0.264680,0.433869
4,2026-01-31T17:30:00Z,Chelsea FC,West Ham United FC,0.630500,0.210900,0.158600
...,...,...,...,...,...,...
146,2026-05-24T15:00:00Z,Liverpool FC,Brentford FC,0.564042,0.197011,0.228989
147,2026-05-24T15:00:00Z,Manchester City FC,Aston Villa FC,0.575068,0.215298,0.205142
148,2026-05-24T15:00:00Z,Nottingham Forest FC,AFC Bournemouth,0.417804,0.236658,0.343129
149,2026-05-24T15:00:00Z,Tottenham Hotspur FC,Everton FC,0.419584,0.257934,0.321449


In [43]:
df_final_probabilities["homeTeam"].unique()

array(['Everton FC', 'Brighton & Hove Albion FC', 'Leeds United FC',
       'Wolverhampton Wanderers FC', 'Chelsea FC', 'Liverpool FC',
       'Aston Villa FC', 'Manchester United FC', 'Nottingham Forest FC',
       'Tottenham Hotspur FC', 'Sunderland AFC', 'AFC Bournemouth',
       'Arsenal FC', 'Burnley FC', 'Fulham FC', 'Newcastle United FC',
       'West Ham United FC', 'Crystal Palace FC', 'Manchester City FC',
       'Brentford FC'], dtype=object)

In [44]:
name_map = {
    "Aston Villa FC": "Aston Villa",
    "Brighton & Hove Albion FC": "Brighton & Hove Albion",
    "AFC Bournemouth": "AFC Bournemouth",   # keep as is
    "Bournemouth": "AFC Bournemouth",
    "Sunderland AFC": "Sunderland",
    "Newcastle United FC": "Newcastle United",
    "Manchester City FC": "Manchester City",
    "Manchester United FC": "Manchester United",
    "West Ham United FC": "West Ham United",
    "Wolverhampton Wanderers FC": "Wolverhampton Wanderers",
    "Tottenham Hotspur FC": "Tottenham Hotspur",
    "Crystal Palace FC": "Crystal Palace",
    "Brentford FC": "Brentford",
    "Everton FC": "Everton",
    "Leeds United FC": "Leeds United",
    "Chelsea FC": "Chelsea",
    "Liverpool FC": "Liverpool",
    "Nottingham Forest FC": "Nottingham Forest",
    "Burnley FC": "Burnley",
    "Fulham FC": "Fulham",
    "Arsenal FC": "Arsenal"
}

df_final_probabilities["home_team_norm"] = df_final_probabilities["homeTeam"].replace(name_map)
df_final_probabilities["away_team_norm"] = df_final_probabilities["awayTeam"].replace(name_map)

premierleague["team_norm"] = premierleague["team"].replace({
    "Brighton & Hove Albion": "Brighton & Hove Albion",
    "AFC Bournemouth": "AFC Bournemouth"
})


[01/26/26 10:11:46] WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_11340\2477332962.py:25:    ]8;id=460485;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=350144;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             SettingWithCopyWarning:                                                               
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_final_probabilities["home_team_norm"] =                                          
                             df_final_probabilities["homeTeam"].replace(name_map)                                  
                                                                                                                   

                    WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_11340\2477332962.py:26:    ]8;id=869731;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=210437;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             SettingWithCopyWarning:                                                               
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_final_probabilities["away_team_norm"] =                                          
                             df_final_probabilities["awayTeam"].replace(name_map)                                  
                                                                                                                   

In [45]:
df_simulation = df_final_probabilities.copy()

In [46]:
# Normalize probabilities so they sum to 1
prob_cols = ["p_home_final", "p_draw_final", "p_away_final"]
df_simulation[prob_cols] = df_simulation[prob_cols].div(df_simulation[prob_cols].sum(axis=1), axis=0)

In [47]:
df_simulation.head()

,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final,home_team_norm,away_team_norm
0,2026-01-26T20:00:00Z,Everton FC,Leeds United FC,0.371258,0.301594,0.327148,Everton,Leeds United
1,2026-01-31T15:00:00Z,Brighton & Hove Albion FC,Everton FC,0.517599,0.258048,0.224353,Brighton & Hove Albion,Everton
2,2026-01-31T15:00:00Z,Leeds United FC,Arsenal FC,0.153816,0.231198,0.614986,Leeds United,Arsenal
3,2026-01-31T15:00:00Z,Wolverhampton Wanderers FC,AFC Bournemouth,0.301451,0.264680,0.433869,Wolverhampton Wanderers,AFC Bournemouth
4,2026-01-31T17:30:00Z,Chelsea FC,West Ham United FC,0.630500,0.210900,0.158600,Chelsea,West Ham United


## 8. Run simulations to build the Premier League table probabilities

In [48]:
def simulate_once(fixtures, table):
    table_sim = table.copy()

    # Use normalized team name column
    points = dict(zip(table_sim["team_norm"], table_sim["pts"]))

    for _, row in fixtures.iterrows():
        home = row["home_team_norm"]
        away = row["away_team_norm"]

        # choose outcome
        probs = [row["p_home_final"], row["p_draw_final"], row["p_away_final"]]
        outcome = np.random.choice(["H", "D", "A"], p=probs)

        if outcome == "H":
            points[home] += 3
        elif outcome == "D":
            points[home] += 1
            points[away] += 1
        else:
            points[away] += 3

    result_df = table_sim.copy()
    result_df["pts"] = result_df["team_norm"].map(points)

    # sort by points and goal difference
    result_df = result_df.sort_values(["pts", "gd"], ascending=[False, False])
    result_df["position"] = np.arange(1, len(result_df)+1)

    return result_df


def run_simulations(fixtures, table, n_sim=10000):
    position_counts = {team: np.zeros(len(table)) for team in table["team_norm"]}

    for _ in range(n_sim):
        final_table = simulate_once(fixtures, table)

        for _, row in final_table.iterrows():
            position_counts[row["team_norm"]][row["position"]-1] += 1

    pos_df = pd.DataFrame(position_counts, index=np.arange(1, len(table)+1))
    pos_df.index.name = "position"
    return pos_df

In [49]:
# RUN
position_distribution = run_simulations(df_simulation, premierleague, n_sim=20000)

In [50]:
position_distribution.index.name = "TEAM"
position_distribution_t = position_distribution.T

In [51]:
position_distribution_pct = position_distribution_t.div(
    position_distribution_t.sum(axis=1),
    axis=0
) * 100


## 9. Preview and present the results graphically

In [52]:
# Build label mapping: "position  team" (extra space for 1-9)
team_labels = (
    premierleague[["team", "position"]]
    .set_index("team")["position"]
    .map(lambda pos: f"{pos}{'  ' if pos < 10 else ' '}")
)

# Join position and team name into one label
team_labels = (
    premierleague[["team", "position"]]
    .assign(
        label=lambda df: df.apply(
            lambda r: f"{r['position']}{'&nbsp;&nbsp;&nbsp;&nbsp;' if r['position'] < 10 else '&nbsp;&nbsp;'}{r['team']}",
            axis=1
        )
    )
    .set_index("team")["label"]
)


# Apply labels to your table index
position_distribution_pct.index = position_distribution_pct.index.map(team_labels)

# Drop position column if present
position_distribution_pct = position_distribution_pct.drop(columns=["position"], errors="ignore")

# Remove index name
position_distribution_pct.index.name = None


In [73]:
greens = plt.cm.Greens
green_cmap = LinearSegmentedColormap.from_list(
    "Greens_soft",
    greens(np.linspace(0.03, 0.65, 256))
)

vmax = 25

def zero_style(val):
    if val < 0.005:
        return "background-color: white !important;"
    return ""

# ---- transform ONLY for colouring ----
color_data = position_distribution_pct.copy()
color_data = (color_data / vmax).pow(0.65) * vmax

position_distribution_pct.style \
    .background_gradient(
        cmap=green_cmap,
        vmin=0,
        vmax=vmax,
        gmap=color_data,
        axis=None          # 🔑 THIS FIXES THE ERROR
    ) \
    .applymap(zero_style) \
    .format("{:.2f}%") \
    .set_table_styles([
        {"selector": "th", "props": [
            ("background-color", "#e6edf4"),
            ("color", "#333"),
            ("text-align", "center"),
            ("font-family", "Inter, Roboto, Arial, sans-serif"),
            ("font-size", "13px"),
            ("font-weight", "600")
        ]},

        {"selector": "th.col_heading", "props": [
            ("text-align", "center")
        ]},

        {"selector": "th.row_heading", "props": [
            ("text-align", "left"),
            ("font-size", "13px"),
            ("font-weight", "600"),
            ("white-space", "nowrap"),
            ("max-width", "250px"),
            ("overflow", "hidden"),
            ("text-overflow", "ellipsis")
        ]},

        {"selector": "tr:nth-child(odd) th.row_heading", "props": [
            ("background-color", "#fbfcfe")
        ]},
        {"selector": "tr:nth-child(even) th.row_heading", "props": [
            ("background-color", "#e6edf4")
        ]},

        {"selector": "td", "props": [
            ("text-align", "center"),
            ("font-family", "Inter, Roboto, Arial, sans-serif"),
            ("font-size", "12px"),
            ("font-weight", "500"),
            ("color", "#000")
        ]}
    ])


[01/26/26 10:42:01] WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_11340\2330607219.py:26:    ]8;id=670326;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=50376;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             FutureWarning: Styler.applymap has been deprecated. Use Styler.map                    
                             instead.                                                                              
                               .applymap(zero_style) \                                                             
                                                                                                                   

TEAM,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1 Arsenal,76.89%,19.52%,3.33%,0.24%,0.01%,0.01%,0.00%,0.01%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2 Manchester City,19.32%,54.03%,20.86%,4.40%,1.09%,0.22%,0.07%,0.01%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
3 Aston Villa,3.62%,21.94%,49.98%,15.78%,5.89%,1.80%,0.66%,0.21%,0.06%,0.03%,0.01%,0.01%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
4 Manchester United,0.01%,0.33%,2.58%,11.76%,17.59%,20.68%,15.05%,10.79%,7.53%,5.19%,3.60%,2.30%,1.28%,0.83%,0.36%,0.11%,0.01%,0.00%,0.00%,0.00%
5 Chelsea,0.10%,2.04%,11.40%,29.41%,24.80%,14.51%,8.21%,4.33%,2.27%,1.33%,0.85%,0.36%,0.23%,0.12%,0.03%,0.01%,0.00%,0.00%,0.00%,0.00%
6 Liverpool,0.06%,1.99%,10.37%,28.41%,24.43%,15.79%,8.25%,4.56%,2.94%,1.57%,0.76%,0.42%,0.27%,0.11%,0.06%,0.01%,0.01%,0.00%,0.00%,0.00%
7 Fulham,0.00%,0.03%,0.21%,1.24%,3.69%,7.35%,10.40%,12.14%,12.55%,12.64%,10.97%,9.52%,7.41%,5.63%,3.64%,1.83%,0.69%,0.05%,0.00%,0.00%
8 Brentford,0.00%,0.05%,0.33%,2.48%,6.21%,10.50%,14.25%,14.13%,12.77%,10.88%,9.28%,7.08%,5.32%,3.28%,2.13%,1.01%,0.27%,0.03%,0.00%,0.00%
9 Newcastle United,0.00%,0.04%,0.61%,3.45%,8.08%,12.37%,15.35%,14.61%,12.21%,10.02%,7.71%,5.92%,4.34%,2.81%,1.47%,0.69%,0.30%,0.03%,0.00%,0.00%
10 Sunderland,0.00%,0.01%,0.07%,0.63%,1.62%,3.46%,5.66%,8.18%,9.71%,11.21%,12.55%,12.31%,11.28%,9.74%,7.24%,4.29%,1.98%,0.08%,0.01%,0.00%
